In [76]:
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt, snoop, matplotlib
from pprint import pprint as print

matplotlib.style.use("seaborn-whitegrid")
pd.set_option("display.width", 5000)
pd.set_option("display.max_columns", 60)
plt.rcParams["figure.figsize"] = (15, 10)

% load_ext snoop
% config InlineBackend.figure_format = 'retina'
# %config InlineBackend.figure_format = 'svg' 
# show all output not only the last result
from IPython.core.interactiveshell import InteractiveShell
from sklearn.linear_model import LinearRegression

InteractiveShell.ast_node_interactivity = "all"
import warnings

warnings.filterwarnings('ignore')

UsageError: Line magic function `%` not found.


In [81]:
from scipy.stats import chi2_contingency
from sklearn.feature_selection import chi2

Все данные вымышленные. Может быть так, что займу не предшествует заявка.

В датасете содержится информация о заявках и займах.
Описание колонок:
time - время подачи заявки или время оформления займа
type - тип записи в таблице (заявка или займ)
amount - сумма заявки/займа
target - целевая перенная
feature1 - какой-то признак
feature2 - какой-то признак

Задача:
Нужно проверить гипотезы о значимости признаков feature1 и feature2 (понять дают ли эти признаки какую-либо информацию для целевой переменной).
Нужно написать код для генерации следующих колонок:
events_ordinal_number - порядковый номер события
second_event_time - время второго события
loan_ordinal_number - порядковый номер займа. Считается как для займов так и для заявок. Заявка всегда имеет номер займа, на который она подается. Если перед получением первого займа было 5 заявок, то все они будут иметь значение признака 1
events_time_diff - разница во времени между событиями
previous_loans_max_amount - максимальная сумма предыдущего займа.

Все признаки считаются для уникальных user_id. Если возникнут вопросы, то можно посмотреть значение признаков в таблице для одного user_id.

In [71]:
df=pd.read_csv("/Users/glebsokolov/Downloads/test.csv")
df

,time,user_id,type,amount,target,feature1,feature2,events_ordinal_number,second_event_time,loan_ordinal_number,events_time_diff,previous_loans_max_amount
0,2001-01-01 00:00:00,443,loanRequest,17017,NaN,1,1,1,2001-11-22 23:00:00,1,NaN,NaN
1,2001-01-01 01:00:00,920,loanRequest,13304,NaN,1,0,1,2002-02-16 08:00:00,1,NaN,NaN
2,2001-01-01 02:00:00,416,loanRequest,13047,NaN,1,0,1,2002-02-18 10:00:00,1,NaN,NaN
3,2001-01-01 03:00:00,382,loanRequest,14201,NaN,0,1,1,2002-02-17 08:00:00,1,NaN,NaN
4,2001-01-01 04:00:00,258,loan,14621,0.0,0,1,1,2001-12-03 11:00:00,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2002-02-21 11:00:00,738,loanRequest,14195,NaN,0,0,8,2002-02-21 11:00:00,1,2 days 02:00:00,NaN
9996,2002-02-21 12:00:00,881,loan,13940,0.0,0,1,14,2002-02-21 12:00:00,2,52 days 15:00:00,14265.0
9997,2002-02-21 13:00:00,72,loan,14325,1.0,1,1,11,2002-02-21 13:00:00,3,27 days 12:00:00,16306.0
9998,2002-02-21 14:00:00,563,loan,15718,0.0,1,0,12,2002-02-21 14:00:00,5,3 days 15:00:00,17170.0


In [84]:
chi2(df[['feature1', 'feature2']], df['target'].fillna(3)
     )

(array([143.02860867,   0.61853828]), array([8.744445e-32, 7.339832e-01]))

In [85]:
# df=df.dropna()
tab = pd.crosstab([df.feature1, df.feature2], df['target'].fillna(3), margins=True)
chi2_contingency(tab)

(359.03172021970863,
 1.7409362560969894e-69,
 12,
 array([[  216.935 ,   198.9748,  1602.0902,  2018.    ],
        [  213.065 ,   195.4252,  1573.5098,  1982.    ],
        [  320.565 ,   294.0252,  2367.4098,  2982.    ],
        [  324.435 ,   297.5748,  2395.9902,  3018.    ],
        [ 1075.    ,   986.    ,  7939.    , 10000.    ]]))

In [73]:
tab

target              0.0  1.0   All
feature1 feature2                 
0        0          329   95   424
         1          322  100   422
1        0          203  383   586
         1          221  408   629
All                1075  986  2061

In [79]:
df[df['user_id'] == 481]

,time,user_id,type,amount,target,feature1,feature2,events_ordinal_number,second_event_time,loan_ordinal_number,events_time_diff,previous_loans_max_amount
144,2001-01-07 00:00:00,481,loan,15501,0.0,0,1,1,2002-02-21 15:00:00,1,NaN,NaN
183,2001-01-08 15:00:00,481,loan,13862,1.0,1,0,2,2002-02-21 15:00:00,2,1 days 15:00:00,15501.0
1978,2001-03-24 10:00:00,481,loanRequest,15593,NaN,1,1,3,2002-02-21 15:00:00,3,74 days 19:00:00,15501.0
2497,2001-04-15 01:00:00,481,loanRequest,15061,NaN,0,0,4,2002-02-21 15:00:00,3,21 days 15:00:00,15501.0
3034,2001-05-07 10:00:00,481,loan,15924,1.0,1,0,5,2002-02-21 15:00:00,3,22 days 09:00:00,15501.0
4378,2001-07-02 10:00:00,481,loanRequest,14485,NaN,0,0,6,2002-02-21 15:00:00,4,56 days 00:00:00,15501.0
4518,2001-07-08 06:00:00,481,loan,14727,0.0,0,0,7,2002-02-21 15:00:00,4,5 days 20:00:00,15924.0
4745,2001-07-17 17:00:00,481,loanRequest,16143,NaN,1,0,8,2002-02-21 15:00:00,5,9 days 11:00:00,15924.0
6100,2001-09-12 04:00:00,481,loanRequest,13933,NaN,0,1,9,2002-02-21 15:00:00,5,56 days 11:00:00,15924.0
6208,2001-09-16 16:00:00,481,loan,14650,1.0,0,0,10,2002-02-21 15:00:00,5,4 days 12:00:00,15924.0


In [80]:
df.insert(loc=df.shape[1], column='eon', value=np.zeros(df.shape[0]))

In [ ]:
df['eon']